In [1]:
!pip install datasets openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

In [3]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("iamjoon/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset['train'].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [ ]:
# 임의 샘플링
df = df[1000:1500].reset_index(drop=True)

In [ ]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
# 프롬프트에 검색 문서 넣는 연습
for idx, s in enumerate(samples[5]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:정부의 8·28 부동산 활성화 대책에 대한 기대로 은행 가계대출이 사상 최고치를 경신했다.13일 한국은행에 따르면 10월 말 예금은행 가계대출 잔액은 474조4000억원으로 전달보다 2조5000억원 증가했다. 지난 6월 4조8000억원 늘어난 뒤 증가세가 잠시 주춤했지만 지난달 다시 큰 폭으로 늘어났다. 은행이 주택금융공사로 넘긴 모기지론 양도분까지 포함하면 10월 한 달간 은행 가계대출은 2조8000억원 급증했다.김정현 한은 금융시장팀 차장은 “취득세 영구 인하 소급 적용에 대한 기대로 주택 거래량이 크게 늘어나면서 주택담보대출(모기지론 양도 포함)이 2조1000억원 늘었다”고 설명했다. 서울시 아파트 거래량은 8월 2800가구에 그쳤으나 9월 4200가구에 이어 10월 7600가구로 증가했다.마이너스통장대출 등 기타 대출도 7000억원 늘었다. 지난 9월 추석 상여금이 통장에 들어오면서 기타 대출은 1000억원 줄었으나 지난달 다시 증가세로 돌아섰다. 한편 10월 은행의 기업대출 증가세는 둔화됐다. 9월 증가액은 5조8000억원이었으나 10월에는 4조1000억원에 그쳤다. 대기업이 1조3000억원, 중소기업이 2조8000억원 각각 늘었다.
---
문서2:박근혜 대통령이 최근 담보 위주의 대출만 취급하는 ‘금융권 보신주의’를 지적하면서 은행들의 중소기업 대출 실적에 관심이 모아지고 있다. 원금과 이자 연체 등 부실 대출이 많았던 국민은행과 우리은행이 건전성 강화 등을 이유로 몸을 사리는 동안, 농협은행과 신한은행이 공격적인 영업으로 치고 나간 모양새다. KT ENS 협력사에 대한 부실대출 등으로 올초 몸살을 앓았던 하나은행도 뒤늦게 중소기업 대출 확대에 나서고 있다.○국민·우리 대출 증가폭 최저국민은행의 중소기업 대출 잔액은 지난달 말 기준으로 68조4155억원이었다. 지난해 말보다 0.9% 늘어나는 데 그쳤다. 우리은행은 59조1265억원으로 같은 기간 증가율이 1.5%에 불과했다. 통상 대출자산 증가율 목표치를 국내총생산(GDP) 증가율보다 조금 

In [ ]:
user_prompts = []
for sample in samples:
  user_prompt = ''
  for idx, s in enumerate(sample):
    user_prompt = user_prompt + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
system_prompt = """당신은 주어진 5개의 검색 문서로부터 답변이 불가능한 질문 5개를 생성해야 합니다.

### 구체적인 지시사항
1. 문서를 5개 드리겠습니다. 각 문서들과 연관된 질문을 하나씩 생성하십시오. 문서1, 문서2, 문서3, 문서4, 문서5 순서대로 합니다.
2. 질문은 각 문서와 연관된 질문이어야 합니다.
3. 답변의 순서는 연관 문서:, 질문:, 판단: 이 세 가지이며 5개의 문서가 주어지므로 총 5번 써야합니다.
4. 중요한 점은 생성된 각 질문은 5개의 그 어떠한 문서로도 답변이 불가능해야 한다는 것입니다.
5. 질문 작성 후에는 추가적인 코멘트는 작성하지 마십시오.
6. 답변의 형식은 다음과 같습니다. 연관 문서에서 질문을 생성하는데 참고한 문서를 적고 그 다음에 질문: 다음에 실제 질문을 생성하세요.
7. 판단: 과정에서 각 질문이 정말로 5개의 문서로 답변이 불가능한지 다시 한 번 판단하고 특정 문서로 답변이 가능한 것으로 판단되면 => False를 적고 답변이 불가능한게 맞다면 => True를 적으세요.
8. => False와 => True 다음에 추가적인 코멘트는 금지합니다.
9. 질문은 짧게 작성하십시오. 예를 들어보겠습니다.
###
ex) 질문: 원종이 설립된 이후 해인사에서 어떤 특별한 행사나 의식이 처음으로 개최되었는지 알려주시겠습니까?
위와 같이 길게 작성하지 마십시오.
ex) 질문: 원종이 설립되고 해인사에서 처음 개최된 행사를 알려줘
위와 같이 짧게 작성하십시오.
###
10. 각 질문은 존댓말과 반말을 교차하면서 사용하십시오. 반말을 했다면 다음 질문은 가급적 존댓말을 하는 식입니다. 최대한 지켜주십시오.
11. 답변 형식은 다음과 같스빈다.
ex)
답변:
연관 문서: 문서1
질문: (문서1과 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 문서 3에 연관된 내용 ~~가 있어서 답변이 가능합니다. => False

연관 문서: 문서2
질문: (문서2와 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

...중략...

연관 문서: 문서5
질문: (문서5와 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True
"""

In [ ]:
from tqdm import tqdm
import openai

In [ ]:
client = openai.OpenAI(api_key="")

In [ ]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompts[:5]):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:26<00:00,  5.27s/it]


In [ ]:
print(result_lst[0])

연관 문서: 문서1
질문: 일감스님의 첫 번째 책 제목은 뭐야?
판단: 해당 질문은 문서 1에 이미 명시되어 있습니다. => False

연관 문서: 문서2
질문: 이용구와 송병준은 원종의 어떤 활동에 기여했나요?
판단: 해당 질문은 문서 2에 이미 명시되어 있습니다. => False

연관 문서: 문서3
질문: 1박2일 인문여행 프로그램 초기에 가장 많이 방문한 유적지는 어디인가요?
판단: 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서4
질문: 2015년 부처님오신날 봉축 법어에서 발표된 주요 메시지는 무엇이었나요?
판단: 해당 질문은 문서 4에 이미 명시되어 있습니다. => False

연관 문서: 문서5
질문: 이번 대법회에서 진제 종정이 발표한 법문 주요 내용은 무엇인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
print(result_lst[1])

연관 문서: 문서1
질문: ‘위대한 탈출’의 도입 부분에서 누락된 구체적인 내용이 무엇인지 알려줘
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서2
질문: 디턴 교수가 빈곤 연구를 시작하게 된 계기는 무엇인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3
질문: 인도의 경제 성장 과정에서 주요 정책 변화는 무엇이었는지 알아?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서4
질문: 로버트 실러 교수가 받은 다른 중요한 조언도 있나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서5
질문: 폴 크루그먼 교수가 최근에 쓴 칼럼 주제는 무엇인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
print(result_lst[2])

연관 문서: 문서1
질문: 허빅-아로 천체와 관련된 천문학자들이 발견한 마지막 천체는 무엇이었는지 알려줘.
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서2
질문: 최초로 고용된 여성 계산수 윌리어미나 플레밍의 고향은 어디였나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3
질문: 유로파 탐사에서 사용될 계획이었던 드릴의 정확한 구경은 얼마죠?
판단: 해당 질문은 문서 1에 구경 관련 설명이 있지만 유로파 드릴의 구경에 대한 언급이 없으므로 답변이 불가능합니다. => True

연관 문서: 문서4
질문: 닐 슈빈이 이빨 화석을 발견한 정확한 날짜는 언제인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서5
질문: 셜록 홈즈가 아편굴에서 만난 다른 정보원은 누구였나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
print(result_lst[3])

연관 문서: 문서1
질문: NXH35C120L2C2E 모듈의 납땜 가능 핀 수는 몇 개야?
판단: 해당 질문은 문서 3에 연관된 내용에서 "납땜" 관련 정보가 없기 때문에 답변이 불가능합니다. => True

연관 문서: 문서2
질문: ST마이크로일렉트로닉스의 EVL400W-EUPL7 평가 보드의 출시 가격은 얼마입니까?
판단: 해당 질문은 문서 2에서 이미 가격이 제공되었기 때문에 답변이 가능합니다. => False

연관 문서: 문서3
질문: LT8491의 I2C 인터페이스를 사용하는 방법을 알려줄래?
판단: 해당 질문은 문서 1, 2, 4, 5에서 관련 정보를 찾을 수 없는 질문입니다. => True

연관 문서: 문서4
질문: XDPL8219 플라이 백 컨트롤러의 BJT 트랜지스터 모델은 무엇인가요?
판단: 해당 질문은 5개 문서 모두에서 BJT 트랜지스터에 대한 언급이 없으므로 답변이 불가능합니다. => True

연관 문서: 문서5
질문: TLE9012AQU의 셀 밸런싱 알고리즘은 어떤 원리로 동작하나요?
판단: 해당 질문은 문서 1, 2, 3, 4에서 관련 정보를 찾을 수 없으므로 답변이 불가능합니다. => True


In [ ]:
print(result_lst[4])

연관 문서: 문서1
질문: 코스피지수가 급락한 날 외국인 투자자들이 순매도한 특정 종목을 구체적으로 알려줘.
판단: 해당 질문은 문서1에 일부 특정된 정보가 있으나 구체적인 목록은 제공되지 않습니다. => True

연관 문서: 문서2
질문: CJ E&M의 작년 4분기 손실의 구체적인 일회성 비용 항목을 알려주실 수 있나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3
질문: 버냉키 쇼크 이후에 각 기업의 구체적인 회사채 발행 주관사를 모두 알려줘.
판단: 해당 질문은 문서3에 일부 특정된 정보가 있으나 모든 주관사 목록은 제공되지 않습니다. => True

연관 문서: 문서4
질문: 넥슨이 중국에서 텐센트와의 계약을 맺은 정확한 시기를 알려주시겠습니까?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서5
질문: 삼성증권의 IB본부가 축소되기 이전에 맡았던 주요 IPO 프로젝트를 알려줘.
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompts):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 500/500 [52:22<00:00,  6.29s/it]


In [ ]:
import datasets
from huggingface_hub import login

In [ ]:
# API 토큰으로 로그인 (발급받은 토큰을 입력)
login("")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
df['no_answer_question'] = result_lst

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("klue-mrc-no-answer-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/iamjoon/klue-mrc-no-answer-question/commit/9f65e20973be4cb2a89defcc097dc5525b670d0e', commit_message='Upload dataset', commit_description='', oid='9f65e20973be4cb2a89defcc097dc5525b670d0e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,no_answer_question
0,"“금강경, 중학생도 읽을 수 있게 풀어썼죠”",문화/TV,hankyung,“‘금강경’은 저에게도 늘 어려웠어요. 우리 불교가 관념적 해석에만 치우쳐 대중에게...,대중이 불교에 친숙해질 수 있도록 일감 스님이 주최한 행사의 이름은?,1,False,‘내비둬 콘서트’,147,[이러한 배경 속에 1908년 3월에 불교계 대표 52인이 한성부 원흥사에 모여서 ...,연관 문서: 문서1\n질문: 금강경 해설서에서 배타적 상호작용의 예시는 무엇입니까?...
1,“디턴 불평등論 왜곡 없었다”...‘위대한 탈출’ 번역 출판한 한경BP 밝혀,종합,hankyung,올해 노벨 경제학상 수상자로 선정된 앵거스 디턴 미국 프린스턴대 교수의 저서 ‘위대...,올해 노벨 경제학상 수상자의 저서와 공통된 소재를 다룬 책은?,2,False,피케티의 ‘21세기 자본’,718,[“가난한 이들이 어떻게 행동하는지 연구하고 어떻게 하면 그들이 좋은 삶을 누릴 수...,연관 문서: 문서1\n질문: 디턴 교수의 책에 원문과 가장 큰 차이가 있는 장은 어...
2,허빅-아로천체,None,wikipedia,"허빅-아로천체는 19세기 말에 버넘이 처음 발견했는데, 그가 릭 관측소에서 91 c...","버넘이 허빅-아로천체를 찾아냈을 때, 사용한 물건은?",1,False,91 cm 구경의 굴절망원경,42,[하버드의 계산수들(Harvard Computers)은 미국 천문학자 에드워드 찰스...,연관 문서: 문서1\n질문: 허빅과 아로가 애리조나주의 천문학 학회에서 처음 만났을...
3,"온세미컨덕터, 산업용 모터 드라이브 애플리케이션 포트폴리오 확대",테크,acrofan,온세미컨덕터(Nasdaq: ON)가 기업들이 특정 설계 문제를 해결하도록 돕기 위해...,NCP730이 동작하는 전압은 최대 몇 볼트인가?,1,False,38V,1503,"[ST마이크로일렉트로닉스(STMicroelectronics, 이하 ST)가 EVL4...",연관 문서: 문서1\n질문: CEO가 TM-PIM 제품 출시를 발표한 프레젠테이션 ...
4,원高·삼성 ‘쇼크’… 증시 불안한 출발,종합,hankyung,코스피지수가 44.15포인트 떨어지는 등 새해 첫 거래일 한국 증시가 호된 신고식을...,13%가량 자사 예상 주가를 줄인 기업은?,1,False,BNP파리바,394,[상장사 10곳 중 6곳의 올 1분기 영업이익 전망치가 지난달 말보다 하향 조정됐다...,연관 문서: 문서1\n질문: 삼성전자의 올해 상반기 실적 전망치는 어떻게 되나요?\...
...,...,...,...,...,...,...,...,...,...,...,...
495,볼펜,None,wikipedia,볼펜은 일회용 모델과 리필 모델로 생산된다. 리필을 사용하면 볼펜 및 소켓을 포함한...,일부 볼펜에서 잉크가 비교적 빨리 건조되기 위해 사용되는 것은?,2,False,하이브리드 잉크 배합물,530,[볼펜 잉크는 일반적으로 염료가 25-40 % 정도 포함 된 페이스트이다. 염료는 ...,연관 문서: 문서1\n질문: Fisher Space Pen은 몇 년도에 최초로 개발...
496,뇌전증,None,wikipedia,"뇌전증 발작은 뇌피질세포의 기능 이상에 의해 발생하는 증상이므로, 신경세포의 기능 ...",뇌전증 발작은 어떤 세포의 기능적 문제로 발생하는 증상인가요?,1,False,뇌피질세포,8,"[간질(癎疾) 또는 전간(癲癎)이라고도 하며, 과거에는 간질이 정식 명칭으로 사용되...",연관 문서: 문서1\n질문: 무균성 뇌수막염의 장기적인 예후를 자세히 설명해줄 수 ...
497,뇌전증,None,wikipedia,"간질(癎疾) 또는 전간(癲癎)이라고도 하며, 과거에는 간질이 정식 명칭으로 사용되었...",뇌전증이 가장 많이 발병하는 때는 언제까지인가?,1,False,생후 1년,688,"[뇌전증 발작은 뇌피질세포의 기능 이상에 의해 발생하는 증상이므로, 신경세포의 기능...",연관 문서: 문서1\n질문: 뇌전증이 아닌 다른 신경계 질환으로 인해 수술이 필요한...
498,힐튼과 맞먹는 에어비앤비 기업가치,국제,hankyung,세계 최대 숙박 공유 서비스기업 에어비앤비가 최근 15억달러(약 1조6800억원) ...,기업 가치가 가장 큰 비상장 기업의 이름은?,1,False,샤오미,292,[공정거래위원회가 자산 5조원 이상 대기업집단의 계열사 중 자산 100억원 미만 비...,연관 문서: 문서1\n질문: 에어비앤비가 최근 자금 조달을 위해 내놓은 지분의 정확...


## 데이터 정제

In [ ]:
import re
import random

In [ ]:
false_samples = []
for sample in df['no_answer_question'].to_list():
  if '=> False' in sample:
    false_samples.append(sample)

print('False가 발생한 샘플의 수:', len(false_samples))

False가 발생한 샘플의 수: 145


In [ ]:
print(df['no_answer_question'].to_list()[15])

연관 문서: 문서1
질문: 경기 오산시 환승터미널 병원 상가의 임대 조건은 어떻게 되나요?
판단: 해당 질문은 문서1에 명확한 답변이 있습니다. => False

연관 문서: 문서2
질문: 노량진역의 민자 역사 개발이 처음 시작된 연도는 언제인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3
질문: 마드리드 지하철 9호선의 6000번대 열차는 어떤 특징이 있나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서4
질문: 서울 지하철 9호선과 관련한 시민 불만 사례 중 일부를 알려주세요.
판단: 해당 질문은 문서4에 실려있는 내용으로 답변이 가능합니다. => False

연관 문서: 문서5
질문: 지하철 9호선 2단계 구간의 총 공사비 예산은 얼마인가요?
판단: 해당 질문은 문서5에 명확한 답변이 있습니다. => False


In [ ]:
import re

def remove_false_decisions(text):
    # 각 질문-판단 세트를 찾는 정규표현식
    pattern = r'(연관 문서: 문서\d+\n질문: .+?\n판단: .+? => (?:True|False))'

    # 모든 세트를 찾아서 리스트로 저장
    sets = re.findall(pattern, text, re.DOTALL)

    # True인 세트만 필터링
    filtered_sets = [s for s in sets if "=> True" in s]

    # 필터링된 세트를 다시 하나의 문자열로 결합
    result = "\n\n".join(filtered_sets)

    return result

In [ ]:
test_result = remove_false_decisions(df['no_answer_question'].to_list()[15])
print(test_result)

연관 문서: 문서2
질문: 노량진역의 민자 역사 개발이 처음 시작된 연도는 언제인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3
질문: 마드리드 지하철 9호선의 6000번대 열차는 어떤 특징이 있나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
def extract_questions(text):
    # 질문을 찾는 정규표현식 패턴
    pattern = r'질문: (.*?)(?=\n판단:)'

    # 모든 질문을 찾아서 리스트로 저장
    questions = re.findall(pattern, text, re.DOTALL)

    # 질문에서 앞뒤 공백 제거
    questions = [q.strip() for q in questions]

    return questions

In [ ]:
# 결과 출력
questions_list = extract_questions(test_result)
print(questions_list)

['노량진역의 민자 역사 개발이 처음 시작된 연도는 언제인가요?', '마드리드 지하철 9호선의 6000번대 열차는 어떤 특징이 있나요?']


In [ ]:
def select_random_question(questions_list):
    if not questions_list:
        return None  # 리스트가 비어있을 경우 None 반환
    return random.choice(questions_list)

In [ ]:
one_questions = []
for five_questions in df['no_answer_question'].to_list():
  true_questions = remove_false_decisions(five_questions)
  questions_list = extract_questions(true_questions)
  random_question = select_random_question(questions_list)
  one_questions.append(random_question)

In [ ]:
one_questions[:5]

['원종 설립 당시 일본 정부의 반응은 어떠했나요?',
 '로버트 실러의 투자 포트폴리오에 사용된 구체적인 주식을 알려줘.',
 '닐 슈빈이 발견한 첫 번째 화석에 대해 더 많은 정보를 알려주시겠습니까?',
 'ST의 CEO가 EVL400W-EUPL7 출시를 위해 어떤 전략을 사용했는지 알려주세요.',
 '중국의 던전앤파이터 인기 게임 순위는 어떻게 되나요?']

In [ ]:
# 5개의 문서를 모두 사용해도 답을 찾을 수 없는 질문
df['no_answer_question'] = one_questions

In [ ]:
# 형식 등의 사유로 질문이 추출되지 않았을 경우
df = df.dropna(subset='no_answer_question').reset_index(drop=True)

In [ ]:
one_questions = df['no_answer_question'].to_list()

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("klue-mrc-no-answer-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/725 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/iamjoon/klue-mrc-no-answer-question/commit/71f76b1709577d14422844bfef2f7ea470f814a6', commit_message='Upload dataset', commit_description='', oid='71f76b1709577d14422844bfef2f7ea470f814a6', pr_url=None, pr_revision=None, pr_num=None)

## 답변 생성

In [ ]:
system_prompt = """당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번, 5번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]], [[ref5]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 답변은 최대한 풍부하게 작성해주시기 바랍니다.
10. 단, 질문에 대한 답이 검색 결과에 없다면 다음과 같이 답변을 작성하십시오. 반드시 아래의 형식을 따라야 합니다.
=> 답변 형식: 검색 결과에는 ~~의 반응을 찾을 수 없습니다.

예시를 드리겠습니다.

예시)
질문: 원종 설립 당시 일본 정부의 반응은 어떠했나요?
답변: 검색 결과에는 원종 설립 당시 일본 정부의 반응을 찾을 수 없습니다.

질문: 중국의 던전앤파이터 인기 게임 순위는 어떻게 되나요?
답변: 검색 결과에는 중국의 던전앤파이터 인기 게임 순위를 찾을 수 없습니다.

검색 결과에 질문에 대한 답변이 없다면 위 형식을 반드시 따르십시오.
"""

In [ ]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
# 프롬프트에 검색 문서 넣는 연습
for idx, s in enumerate(samples[5]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:정부의 8·28 부동산 활성화 대책에 대한 기대로 은행 가계대출이 사상 최고치를 경신했다.13일 한국은행에 따르면 10월 말 예금은행 가계대출 잔액은 474조4000억원으로 전달보다 2조5000억원 증가했다. 지난 6월 4조8000억원 늘어난 뒤 증가세가 잠시 주춤했지만 지난달 다시 큰 폭으로 늘어났다. 은행이 주택금융공사로 넘긴 모기지론 양도분까지 포함하면 10월 한 달간 은행 가계대출은 2조8000억원 급증했다.김정현 한은 금융시장팀 차장은 “취득세 영구 인하 소급 적용에 대한 기대로 주택 거래량이 크게 늘어나면서 주택담보대출(모기지론 양도 포함)이 2조1000억원 늘었다”고 설명했다. 서울시 아파트 거래량은 8월 2800가구에 그쳤으나 9월 4200가구에 이어 10월 7600가구로 증가했다.마이너스통장대출 등 기타 대출도 7000억원 늘었다. 지난 9월 추석 상여금이 통장에 들어오면서 기타 대출은 1000억원 줄었으나 지난달 다시 증가세로 돌아섰다. 한편 10월 은행의 기업대출 증가세는 둔화됐다. 9월 증가액은 5조8000억원이었으나 10월에는 4조1000억원에 그쳤다. 대기업이 1조3000억원, 중소기업이 2조8000억원 각각 늘었다.
---
문서2:박근혜 대통령이 최근 담보 위주의 대출만 취급하는 ‘금융권 보신주의’를 지적하면서 은행들의 중소기업 대출 실적에 관심이 모아지고 있다. 원금과 이자 연체 등 부실 대출이 많았던 국민은행과 우리은행이 건전성 강화 등을 이유로 몸을 사리는 동안, 농협은행과 신한은행이 공격적인 영업으로 치고 나간 모양새다. KT ENS 협력사에 대한 부실대출 등으로 올초 몸살을 앓았던 하나은행도 뒤늦게 중소기업 대출 확대에 나서고 있다.○국민·우리 대출 증가폭 최저국민은행의 중소기업 대출 잔액은 지난달 말 기준으로 68조4155억원이었다. 지난해 말보다 0.9% 늘어나는 데 그쳤다. 우리은행은 59조1265억원으로 같은 기간 증가율이 1.5%에 불과했다. 통상 대출자산 증가율 목표치를 국내총생산(GDP) 증가율보다 조금 

In [ ]:
user_prompts = []
for sample in samples:
  user_prompt = '검색 결과:\n'
  for idx, s in enumerate(sample):
    user_prompt = user_prompt  + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
len(user_prompts)

497

In [ ]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(one_questions, user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [ ]:
print(user_prompt_for_answer[0])

질문:원종 설립 당시 일본 정부의 반응은 어떠했나요?
검색 결과:
문서1:“‘금강경’은 저에게도 늘 어려웠어요. 우리 불교가 관념적 해석에만 치우쳐 대중에게 쉽게 다가가지 못하고 있다는 생각이 들었습니다. 도전적인 해석을 담아 중학교 2학년도 이해할 수 있는 금강경 해설서를 써보자고 마음먹었죠.”전북 김제 금산사에서 휴식형 템플스테이 ‘내비둬 콘서트’를 기획, 진행하는 등 불교 대중화를 위해 앞장서온 일감 스님(사진)이 첫 번째 책을 출간했다. 조계종이 근본경전으로 삼는 ‘금강경’을 대중의 눈높이에 맞춰 풀어 쓴 금강경을 읽는 즐거움(민족사)이다.조계종 총무원 기획실장을 맡고 있는 일감스님은 “공(空)·무아(無我) 등 불교의 어려운 사상과 용어를 쉽게 풀어쓰려고 했다”며 “원전에 없는 글도 많이 담고 원문 자체의 번역을 도전적으로 과감하게 해봤다”고 말했다. 특히 공(空) 사상을 글자 그대로 텅 빈 것, 허무주의로 오해하지 않도록 뜻을 새겼다.일감스님은 ‘세상에 존재하는 모든 것은(一切有爲法·일체유위법) 마치 꿈 같고 허깨비 같고 물거품 같고 그림자 같으며(如夢幻泡影·여몽환포영) 이슬 같고 번개와 같으니(如露亦如電·여로역여전), 마땅히 이와 같이 보아라(應作如是觀·응작여시관)’는 구절을 예로 들었다. “이는 세상이 다 허망하니 다 버리라는 의미가 아니라 그렇기 때문에 더 아름답고 귀하게, 참되게 살라는 의미”라고 설명했다.금강경은 집착에서 벗어나 공의 이치를 깨닫는 것이다. 일감스님은 그중에서도 ‘관계의 미학’을 강조했다. “금강경 전체를 꿰뚫는 한 말씀이 있다면 ‘나와 남이 둘이 아니다. 우리 모두가 관계 속에 존재하니, 관계를 좋게 해서 행복하게 잘 살라’는 것입니다.”일감 스님은 “실제로 수많은 고통이 관계에서 비롯되고 불행감과 행복감, 심지어 온갖 질병이 관계에서 오는 경우가 많다”며 “관계의 미학을 알면 이 시대의 온갖 갈등을 근원부터 해결할 수 있는 길이 열릴 것”이라고 설명했다.이어 “내 옳음 속에도 빈틈이 있고, 상대방의 그름 속에도 이유가 있다

In [ ]:
from tqdm import tqdm
import openai

In [ ]:
client = openai.OpenAI(api_key="")

In [ ]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer[:5]):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:03<00:00,  1.39it/s]


In [ ]:
result_lst

['검색 결과에는 원종 설립 당시 일본 정부의 반응을 찾을 수 없습니다.',
 '검색 결과에는 로버트 실러의 투자 포트폴리오에 사용된 구체적인 주식을 찾을 수 없습니다.',
 '검색 결과에는 닐 슈빈이 발견한 첫 번째 화석에 대한 정보를 찾을 수 없습니다.',
 '검색 결과에는 ST의 CEO가 EVL400W-EUPL7 출시를 위해 사용한 전략에 대한 정보를 찾을 수 없습니다.',
 '검색 결과에는 중국의 던전앤파이터 인기 게임 순위를 찾을 수 없습니다.']

In [ ]:
# 전체 데이터에 대해서 답변을 생성
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 497/497 [09:19<00:00,  1.13s/it]


In [ ]:
df['no_answer'] = result_lst

In [ ]:
# 답이 없기를 기대했지만 추출된 경우
df[df['no_answer'].str.contains('ref')]

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,no_answer_question,no_answer
2,허빅-아로천체,None,wikipedia,"허빅-아로천체는 19세기 말에 버넘이 처음 발견했는데, 그가 릭 관측소에서 91 c...","버넘이 허빅-아로천체를 찾아냈을 때, 사용한 물건은?",1,False,91 cm 구경의 굴절망원경,42,[하버드의 계산수들(Harvard Computers)은 미국 천문학자 에드워드 찰스...,닐 슈빈이 발견한 첫 번째 화석에 대해 더 많은 정보를 알려주시겠습니까?,생물학자 닐 슈빈이 발견한 첫 번째 화석은 그린란드를 탐사하던 중 발견된 매우 작은...
10,"현대·기아차, 올 800만대 판매 ‘청신호’ ...신형 쏘나타·제네시스 앞세워 상반...",산업섹션,hankyung,현대·기아자동차가 해외 시장에서의 판매 호조에 힘입어 올 상반기 세계 시장에서 40...,코란도C는 어느 자동차회사의 차종인가?,1,False,쌍용차,1094,[현대자동차가 신형 제네시스에 들어가는 주요 전장부품의 공급선을 독일 콘티넨탈에서 ...,한불모터스의 채권단 구성이 어떻게 되어 있습니까?,검색 결과에는 한불모터스의 채권단 구성이 신한은행을 중심으로 이루어져 있음을 확인할...
15,한경 매물마당 land.hankyung.com ...서울 대치동 역세권 수익형 빌딩...,부동산,hankyung,○서울 강남 환승역세권 사옥 및 투자용 빌딩=삼성동 지하철 도보 3분 거리 대지 1...,아직 운행하지 않는 지하철은 몇 호선인가?,1,False,9호선,316,[한국철도공사(코레일)가 추진하는 5개 민자 역사 건립 사업이 10년 넘게 표류하고...,마드리드 지하철 9호선의 6000번대 열차는 어떤 특징이 있나요?,마드리드 지하철 9호선의 6000번대 열차의 특징에 대해서는 다음과 같은 정보가 있...
19,소포차니 수도원,None,wikipedia,이 수도원은 삼위일체에게 봉헌되었고 1265년에 내부 장식을 완성하였다. 1263년...,프레스코가 유명한 곳은 언제부터 세르비아 공화국이 보호해주었나요?,2,False,1979년,472,[발칸반도 전선 (제1차 세계 대전)|제1차 세계 대전 기간의 불가리아|세르비아 전...,제1차 세계 대전 당시 세르비아군을 지원한 주요 연합국은 어떤 국가들인가요?,제1차 세계 대전 당시 세르비아군을 지원한 주요 연합국은 프랑스와 영국입니다. 문서...
23,히사쓰 오렌지 철도,None,wikipedia,가고시마현이나 가고시마 현 연선 시정촌의 신칸센이 정차하지 않는 아쿠네시 이외에 재...,"철도보다 자가용 이용이 많아지면서, 히사쓰 오렌지 철도 노선의 대항마로 부상한 것은?",1,False,남큐슈 자동차 전용 도로,522,"[1987년에 일본국유철도의 분할 민영화가 진행되고, 서일본 여객철도는 오사카 권을...",어번 네트워크 구역 지정이 처음 된 연도는 언제인가요?,어번 네트워크 구역 지정이 처음 된 연도는 1988년입니다. 1988년 3월 13일...
...,...,...,...,...,...,...,...,...,...,...,...,...
457,김대중 내란음모 조작 사건,None,wikipedia,"미국으로 출국한 김대중은 일체의 정치 활동을 안하기로 전두환 정부와 약속했으나, 얼...",마르코스 정권이 공항에서 암살한 필리핀 야당 지도자는?,1,False,베니그노 아키노,244,[코라손 아키노 전 필리핀 대통령은 현재 필리핀 대통령인 베니그노 노이노이 아키노의...,라몬 막사이사이 대통령이 국민당 후보로 출마하게 된 계기는 정확히 무엇이었는지 설명...,라몬 막사이사이 대통령이 국민당 후보로 출마하게 된 계기는 필리핀 내 정치적 상황과...
459,"유럽사법재판소 “개인정보, 美 이전 금지” 판결...구글·포드 등 美 기업 ‘초비상’",국제,hankyung,구글과 페이스북 등 미국 기업이 유럽 사용자의 개인정보를 미국에 있는 서버로 보낼 ...,세이프 하버' 무효화의 시발점이 된 내부 고발 사건이 일어난 해는?,1,False,2013년,118,[미국 연방검찰이 억만장자 헤지펀드 매니저인 스티븐 코언 회장(사진)이 이끄는 SA...,조지 로엡 사건에서 토머스는 무슨 혐의로 재판을 받았나요?,조지 로엡 사건에서 토머스는 '흑인 수병 해럴드 J. 맨스필드'의 살해 사건과 관련...
470,이오 (위성),None,wikipedia,이오는 주로 이산화 황(SO)으로 구성된 얇은 대기를 가지고 있다. 이 밖에 일산화...,이오에 착륙하기 위해서 탐사선이 이용하는 것은?,1,False,역추진 로켓,706,"[이오(Io, Ἰώ)는 목성의 위성 중 하나로 갈릴레이 위성에 속하는 위성이다. 지...",갈릴레오 호가 발견한 이오의 모든 지질학적 특징은 무엇인가요?,갈릴레오 호가 이오에서 발견한 모든 지질학적 특징에 대해 살펴보면 여러 중요한 지질...
473,삼단논법,None,wikipedia,삼단논법(三段論法)은 미리 알려진 두 판단에서 그것들과는 다른 하나의 새로운 판단으...,결론에 표시하지 않는 개념은?,1,False,매개념,428,[무변론판결(無辯論判決)이란 피고가 답변서 제출기간 이내에 답변서를 제출하지 아니하...,S와 M 사이의 관계는 무엇인가?,S와 M 사이의 관계는 삼단논법에서의 소개념(S)과 매개념(M) 사이의 관계를 뜻합...


답이 없어야 하는데 추출된 경우에는 삭제합니다. (데이터를 보고 이상하지 않다고 판단되면 사용하셔도 됩니다.)

In [ ]:
df = df[~df['no_answer'].str.contains('ref')].reset_index(drop=True)

In [ ]:
# 이상한 데이터가 없는지 체크
sample_check = []
for sample in df['no_answer'].to_list():
  if '검색 결과' not in sample:
    sample_check.append(sample)

In [ ]:
len(sample_check)

4

In [ ]:
sample_check[3]

'교원그룹이 레이캅 외에 첫 번째로 위탁 판매를 시작한 제품에 대한 정보를 찾을 수 없습니다.'

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("klue-mrc-no-answer-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/763 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/iamjoon/klue-mrc-no-answer-question/commit/1edcf8e0d05cf1aefa58bdd605d1c73d26d0713c', commit_message='Upload dataset', commit_description='', oid='1edcf8e0d05cf1aefa58bdd605d1c73d26d0713c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
print('질문:', df['no_answer_question'].loc[15])

질문: 현재현 동양 회장의 사표가 수리된 이유는 무엇인가요?


In [ ]:
for idx, s in enumerate(samples[15]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:박용만 두산그룹 회장이 21일 대한상공회의소 회장에 취임했다. ‘소통의 달인’이란 별칭에 걸맞게 첫날부터 대한상의 임직원들과 스스럼없이 인사를 주고받으며 취임식 분위기를 주도했다. 그는 경제민주화 등 현안과 관련해선 “기업의 변화를 요구하는 사회의 목소리를 겸허히 듣겠지만 한꺼번에 감당하기는 어려운 만큼 회원사들의 입장을 충실히 대변하겠다”며 강한 어조로 말했다. 박 회장은 취임식에 이어 열린 기자간담회에서 “대한상의 회장으로서 구체적인 활동 계획은 차차 세우겠다”면서도 주요 이슈에 대해서는 조목조목 설명하며 소신있는 자세를 보였다. ○“통상임금 문제로 산업계 공멸 우려”박 회장은 취임사와 간담회에서 최근 기업을 둘러싼 주요 현안을 언급하며 우려를 표시했다. 그는 “저성장 국면 타개, 일자리 창출과 투자확대 등 기업 본연의 일에다 통상임금, 경제민주화 입법, 하도급법 개정, 골목상권 보호와 일감 몰아주기 규제, 상법과 세법 개정, 중소기업의 인력수급 불일치, 대기업과 중소기업의 양극화, 수도권과 지방경제의 양극화 등 다 열거하자면 숨이 찰 정도”라고 지적했다. 박 회장은 특히 통상임금 문제를 주목했다. 그는 “지난 1주일 동안 전국을 돌며 지역 상의 회원들을 만났는데 공통적으로 지적한 이슈가 통상임금”이라며 “산업계가 공멸할 수도 있겠다는 생각이 들 만큼 중요하고 특히 중소기업들은 생사가 걸린 문제로 받아들이고 있었다”고 전했다. 박 회장은 “기업의 억울한 피해를 막기 위해서는 기존에 합의된 임금체계를 존중해야 하며 법원도 경제에 미칠 영향을 감안해 신중한 결정을 내릴 것을 기대한다”고 말했다.기업 경영과 관련한 각종 입법과 규제에 대해서도 의견을 내놨다. 박 회장은 “불공정한 관행을 없애기 위해 경제민주화를 추진하는 것은 시대적 과제라고 인정하지만 그 과정엔 소통이 전제돼야 한다”며 “입법과 규제 단계까지 가지 않고도 토론과 소통으로 유연한 해결책을 마련할 수 있다”고 진단했다. 입법부에 대한 당부도 곁들였다. 박 회장은 “외국인투자촉진법 개정안이 국

In [ ]:
print(df['no_answer'].loc[15])

검색 결과에는 현재현 동양 회장의 사표가 수리된 이유를 찾을 수 없습니다.


In [ ]:
import re

In [ ]:
# 인용 문서 번호 추출
def extract_ref_numbers(text):
    # 정규표현식을 사용하여 [[ref숫자]] 패턴 찾기
    pattern = r'\[\[ref(\d+)\]\]'

    # 모든 매치 찾기
    matches = re.findall(pattern, text)

    # 문자열을 정수로 변환하고 중복 제거 후 정렬
    unique_numbers = sorted(set(map(int, matches)))

    return unique_numbers

In [ ]:
# 테스트
result = extract_ref_numbers(df['no_answer'].loc[15])
print(result)

[]


In [ ]:
extracted_ref_numbers = []
for sample in df['no_answer'].to_list():
  extracted_ref_numbers.append(extract_ref_numbers(sample))

In [ ]:
df['extracted_ref_numbers'] = extracted_ref_numbers

In [ ]:
# 인용 문서가 존재하는지 확인
df[~(df['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,no_answer_question,no_answer,extracted_ref_numbers


In [ ]:
df['search_result'] = samples

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("klue-mrc-no-answer-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/718 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/iamjoon/klue-mrc-no-answer-question/commit/f3b1973cb09534e5eaea5eedc0df3f91f69bd426', commit_message='Upload dataset', commit_description='', oid='f3b1973cb09534e5eaea5eedc0df3f91f69bd426', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
df[['no_answer_question', 'search_result', 'no_answer', 'extracted_ref_numbers']]

,no_answer_question,search_result,no_answer,extracted_ref_numbers
0,원종 설립 당시 일본 정부의 반응은 어떠했나요?,[“‘금강경’은 저에게도 늘 어려웠어요. 우리 불교가 관념적 해석에만 치우쳐 대중에...,검색 결과에는 원종 설립 당시 일본 정부의 반응을 찾을 수 없습니다.,[]
1,로버트 실러의 투자 포트폴리오에 사용된 구체적인 주식을 알려줘.,[올해 노벨 경제학상 수상자로 선정된 앵거스 디턴 미국 프린스턴대 교수의 저서 ‘위...,검색 결과에는 로버트 실러의 투자 포트폴리오에 사용된 구체적인 주식을 찾을 수 없습니다.,[]
2,ST의 CEO가 EVL400W-EUPL7 출시를 위해 어떤 전략을 사용했는지 알려주세요.,[온세미컨덕터(Nasdaq: ON)가 기업들이 특정 설계 문제를 해결하도록 돕기 위...,검색 결과에는 ST의 CEO가 EVL400W-EUPL7 출시를 위해 사용한 전략을 ...,[]
3,중국의 던전앤파이터 인기 게임 순위는 어떻게 되나요?,[코스피지수가 44.15포인트 떨어지는 등 새해 첫 거래일 한국 증시가 호된 신고식...,검색 결과에는 중국의 던전앤파이터 인기 게임 순위를 찾을 수 없습니다.,[]
4,웰컴저축은행이 예신저축은행을 인수하기 전에 다른 인수 후보가 있었나요?,[정부의 8·28 부동산 활성화 대책에 대한 기대로 은행 가계대출이 사상 최고치를 ...,검색 결과에는 웰컴저축은행이 예신저축은행을 인수하기 전에 다른 인수 후보가 있었다는...,[]
...,...,...,...,...
399,Fisher Space Pen은 몇 년도에 최초로 개발되었어?,[볼펜은 일회용 모델과 리필 모델로 생산된다. 리필을 사용하면 볼펜 및 소켓을 포함...,검색 결과에는 Fisher Space Pen이 몇 년도에 최초로 개발되었는지에 대한...,[]
400,기면증 환자가 피해야 할 특정 식품이 있나요?,"[뇌전증 발작은 뇌피질세포의 기능 이상에 의해 발생하는 증상이므로, 신경세포의 기능...",검색 결과에는 기면증 환자가 피해야 할 특정 식품에 대한 정보를 찾을 수 없습니다.,[]
401,틱톡이 추가로 계획하고 있는 다른 안전 기능이 무엇인지 아십니까?,"[간질(癎疾) 또는 전간(癲癎)이라고도 하며, 과거에는 간질이 정식 명칭으로 사용되...",검색 결과에는 틱톡이 추가로 계획하고 있는 다른 안전 기능에 대한 정보를 찾을 수 ...,[]
402,비트코인이 처음 개발될 당시 개발 비용은 얼마였나요?,[세계 최대 숙박 공유 서비스기업 에어비앤비가 최근 15억달러(약 1조6800억원)...,검색 결과에는 비트코인이 처음 개발될 당시의 개발 비용에 대한 정보를 찾을 수 없습니다.,[]
